In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

In [ ]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Train Model

In [ ]:
from classification.models.M5 import M5, M5PLModule

hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.001,
    "lr_decay": 0.95
}

model = M5PLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

In [ ]:
trainer = pl.Trainer(
    max_epochs=2,
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name="M5"),
    gpus=1 if torch.cuda.is_available() else None,
    log_gpu_memory='all'
)

trainer.fit(model)

In [ ]:
#best_model = load_module("jo2", M5PLModule)
model.cuda()
model.report()

### Run a single attack

In [ ]:
from attacks.pgd import ProjectedGradientDescent
attack = ProjectedGradientDescent(model.model, model.train_dataloader(), {"norm":"inf", "epsilon": .01}, early_stopping=-1, device='cuda', save_samples=True)
attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

In [ ]:
from attacks.volume import VolumeAttack
attack = VolumeAttack(model.model, model.train_dataloader(), {"epsilon":0.9}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform
